In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6559642747908894574
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1447745945
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13131042715619732915
physical_device_desc: "device: 0, name: Quadro M1000M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [28]:
np.random.seed(123)
tf.random.set_seed(123)

n_steps=5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(16))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
for idx, (X_batch) in enumerate(dataset):
    print('X_batch:',X_batch)

X_batch: <_VariantDataset shapes: (), types: tf.int32>
X_batch: <_VariantDataset shapes: (), types: tf.int32>
X_batch: <_VariantDataset shapes: (), types: tf.int32>
X_batch: <_VariantDataset shapes: (), types: tf.int32>
X_batch: <_VariantDataset shapes: (), types: tf.int32>
X_batch: <_VariantDataset shapes: (), types: tf.int32>


In [29]:
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
for idx, (X_batch) in enumerate(dataset):
    print('X_batch:',X_batch)

X_batch: tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
X_batch: tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int32)
X_batch: tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int32)
X_batch: tf.Tensor([ 6  7  8  9 10], shape=(5,), dtype=int32)
X_batch: tf.Tensor([ 8  9 10 11 12], shape=(5,), dtype=int32)
X_batch: tf.Tensor([10 11 12 13 14], shape=(5,), dtype=int32)


In [30]:
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for idx, (X_batch, y_batch) in enumerate(dataset):
    print('X_batch:',X_batch)
    print('y_batch:',y_batch)

X_batch: tf.Tensor(
[[10 11 12 13]
 [ 2  3  4  5]
 [ 8  9 10 11]], shape=(3, 4), dtype=int32)
y_batch: tf.Tensor(
[[11 12 13 14]
 [ 3  4  5  6]
 [ 9 10 11 12]], shape=(3, 4), dtype=int32)
X_batch: tf.Tensor(
[[0 1 2 3]
 [4 5 6 7]
 [6 7 8 9]], shape=(3, 4), dtype=int32)
y_batch: tf.Tensor(
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 7  8  9 10]], shape=(3, 4), dtype=int32)


# **CharRNN - Shakespeare_Dataset**

In [32]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
filepath

1122304/1115394 [==============================] - 15s 13us/step


'C:\\Users\\user\\.keras\\datasets\\shakespeare.txt'

In [33]:
with open(filepath) as f:
    shakespeare_txt = f.read()

In [37]:
print(shakespeare_txt[:140])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to 


In [39]:
''.join(sorted(set(shakespeare_txt.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [40]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_txt)

In [42]:
tokenizer.texts_to_sequences(['first'])

[[20, 6, 9, 8, 3]]

In [44]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [46]:
max_id = tokenizer.word_index
max_id

{' ': 1,
 'e': 2,
 't': 3,
 'o': 4,
 'a': 5,
 'i': 6,
 'h': 7,
 's': 8,
 'r': 9,
 'n': 10,
 '\n': 11,
 'l': 12,
 'd': 13,
 'u': 14,
 'm': 15,
 'y': 16,
 'w': 17,
 ',': 18,
 'c': 19,
 'f': 20,
 'g': 21,
 'b': 22,
 'p': 23,
 ':': 24,
 'k': 25,
 'v': 26,
 '.': 27,
 "'": 28,
 ';': 29,
 '?': 30,
 '!': 31,
 '-': 32,
 'j': 33,
 'q': 34,
 'x': 35,
 'z': 36,
 '3': 37,
 '&': 38,
 '$': 39}

In [47]:
max_id = len(max_id)

In [48]:
dataset_size = tokenizer.document_count
dataset_size

1115394

In [90]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_txt]))-1
encoded[:100]

array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 21,  1,
       19,  3,  8,  1,  0, 16,  1,  0, 22,  8,  3, 18,  1,  1, 12,  0,  4,
        9, 15,  0, 19, 13,  8,  2,  6,  1,  8, 17,  0,  6,  1,  4,  8,  0,
       14,  1,  0,  7, 22,  1,  4, 24, 26, 10, 10,  4, 11, 11, 23, 10,  7,
       22,  1,  4, 24, 17,  0,  7, 22,  1,  4, 24, 26, 10, 10, 19,  5,  8,
        7,  2,  0, 18,  5,  2,  5, 35,  1,  9, 23, 10, 15,  3, 13])

In [144]:
train_size = dataset_size*15//100
print(train_size)
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

167309


In [145]:
for x in dataset.take(2):
    print(x)

tf.Tensor(19, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


In [146]:
n_steps=100
windows_length=n_steps+1
batch_size=32
dataset = dataset.repeat().window(windows_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(windows_length))
dataset = dataset.map(lambda window: (window[:-1], window[1:])).shuffle(10000).batch(batch_size)

In [147]:
for x,y in dataset.take(2):
    print('\nX_batch:',x)
    print('Y_batch:',y)


X_batch: tf.Tensor(
[[ 8  5 18 ... 14  5  2]
 [ 2  6  1 ... 12  5 12]
 [ 1  1  0 ...  1  0 19]
 ...
 [12 26  0 ... 13  0  4]
 [15  0 25 ...  3  0 21]
 [ 6  1  0 ...  1 11 22]], shape=(32, 100), dtype=int32)
Y_batch: tf.Tensor(
[[ 5 18  5 ...  5  2 15]
 [ 6  1  0 ...  5 12  0]
 [ 1  0 16 ...  0 19 11]
 ...
 [26  0  6 ...  0  4  8]
 [ 0 25  1 ...  0 21  8]
 [ 1  0  4 ... 11 22  7]], shape=(32, 100), dtype=int32)

X_batch: tf.Tensor(
[[ 3 16  0 ... 12  0  1]
 [ 5  9 24 ...  5 14  1]
 [ 2  5 35 ...  5  9  0]
 ...
 [ 7  1  9 ... 12  0  2]
 [ 0  1  4 ...  1  7  7]
 [ 0  5 14 ...  8 14  7]], shape=(32, 100), dtype=int32)
Y_batch: tf.Tensor(
[[16  0  5 ...  0  1  7]
 [ 9 24  0 ... 14  1  0]
 [ 5 35  1 ...  9  0  8]
 ...
 [ 1  9  4 ...  0  2  6]
 [ 1  4  2 ...  7  7  0]
 [ 5 14 22 ... 14  7 17]], shape=(32, 100), dtype=int32)


In [148]:
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [149]:
for x,y in dataset.take(1):
    print('\nX_batch:',x)
    print('Y_batch:',y)
    print(x.shape, y.shape)


X_batch: tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0

In [150]:
model = keras.models.Sequential([
    keras.layers.GRU(64, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_9 (GRU)                  (None, None, 64)          20160     
_________________________________________________________________
time_distributed_8 (TimeDist (None, None, 39)          2535      
Total params: 22,695
Trainable params: 22,695
Non-trainable params: 0
_________________________________________________________________


In [151]:
with tf.device('/device:GPU:0'):
    history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=2)

Train for 5228 steps
Epoch 1/2
5228/5228 [==============================] - 184s 35ms/step - loss: 2.0332
Epoch 2/2
5228/5228 [==============================] - 170s 33ms/step - loss: 1.8139


In [153]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [156]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
Y_pred

array([[ 1, 13,  0,  2,  9,  1,  0,  2,  3, 13]], dtype=int64)

In [162]:
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'u'

In [138]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [163]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [187]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [166]:
print(complete_text("t", temperature=2))

tycc ba
doveay enerobpance, cra
vijoevins! drazeryp


## Stateful RNN

In [152]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(windows_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(windows_length))
dataset = dataset.repeat().batch(1)
dataset = dataset.map(lambda window: (window[:-1], window[1:]))
dataset = dataset.map(lambda X_batch, y_batch:
                                         (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [168]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(windows_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(windows_length))
    datasets.append(dataset)

dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [177]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(64, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [178]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_16 (GRU)                 (32, None, 128)           64896     
_________________________________________________________________
gru_17 (GRU)                 (32, None, 64)            37248     
_________________________________________________________________
time_distributed_12 (TimeDis (None, None, 39)          2535      
Total params: 104,679
Trainable params: 104,679
Non-trainable params: 0
_________________________________________________________________


In [179]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [181]:
with tf.device('/device:GPU:0'):
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
    steps_per_epoch = train_size // batch_size // n_steps   # Becoz shift=m_steps (train_size/(n_steps*batch_size))
    history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=7,
                        callbacks=[ResetStatesCallback()])

Train for 52 steps
Epoch 1/7
52/52 [==============================] - 48s 924ms/step - loss: 3.6636
Epoch 2/7
52/52 [==============================] - 40s 772ms/step - loss: 3.6636
Epoch 3/7
52/52 [==============================] - 39s 750ms/step - loss: 3.6636
Epoch 4/7
52/52 [==============================] - 38s 735ms/step - loss: 3.6636
Epoch 5/7
52/52 [==============================] - 39s 755ms/step - loss: 3.6636
Epoch 6/7
52/52 [==============================] - 38s 730ms/step - loss: 3.6636
Epoch 7/7
52/52 [==============================] - 38s 729ms/step - loss: 3.6636


In [182]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(64, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [183]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [184]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

# Sentiment Analysis

In [4]:
(X_train, y_test), (X_valid, y_test) = keras.datasets.imdb.load_data()

17465344/17464789 [==============================] - 121s 7us/step


In [54]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [58]:
word_idx = keras.datasets.imdb.get_word_index()
id2word = {id_+ 3 : word for word, id_ in word_idx.items()}
for id_, word in enumerate(('<pad>', '<sos>', '<unk>')):
    id2word[id_]=word

' '.join([id2word[id_] for id_ in X_train[0]])

1646592/1641221 [==============================] - 36s 22us/step


"<sos> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shou

In [5]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

Shuffling and writing examples to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteHI5H61\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteHI5H61\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteHI5H61\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [6]:
datasets.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [7]:
train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples
train_size, test_size

(25000, 25000)

In [10]:
for X_batch, y_batch in datasets['train'].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print('Review: ', review.decode('utf-8')[:250],'.....')
        print('Label: ', label, '\n')

Review:  This is a big step down after the surprisingly enjoyable original. This sequel isn't nearly as fun as part one, and it instead spends too much time on plot development. Tim Thomerson is still the best thing about this series, but his wisecracking is  .....
Label:  0 

Review:  Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on m .....
Label:  0 



In [15]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b'<br\s*/?>', b' ')
    X_batch = tf.strings.regex_replace(X_batch, b'[^a-zA-Z]', b' ')
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value='<pad>'), y_batch

In [16]:
preprocess(X_batch, y_batch)

(<tf.Tensor: id=394, shape=(2, 61), dtype=string, numpy=
 array([[b'This', b'is', b'a', b'big', b'step', b'down', b'after', b'the',
         b'surprisingly', b'enjoyable', b'original', b'This', b'sequel',
         b'isn', b't', b'nearly', b'as', b'fun', b'as', b'part', b'one',
         b'and', b'it', b'instead', b'spends', b'too', b'much', b'time',
         b'on', b'plot', b'development', b'Tim', b'Thomerson', b'is',
         b'still', b'the', b'best', b'thing', b'about', b'this',
         b'series', b'but', b'his', b'wisecracking', b'is', b'toned',
         b'down', b'in', b'this', b'entry', b'The', b'performances',
         b'are', b'all', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
         b'<pad>', b'<pad>'],
        [b'Perhaps', b'because', b'I', b'was', b'so', b'young',
         b'innocent', b'and', b'BRAINWASHED', b'when', b'I', b'saw',
         b'it', b'this', b'movie', b'was', b'the', b'cause', b'of',
         b'many', b'sleepless', b'nights', b'for', b'me', b'I', b'hav

In [17]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [18]:
vocabulary.most_common()[:5]

[(b'<pad>', 223998),
 (b'the', 61156),
 (b'a', 38569),
 (b'of', 33984),
 (b'and', 33432)]

In [19]:
len(vocabulary)

49739

In [35]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]
truncated_vocabulary[:5], len(truncated_vocabulary)

([b'<pad>', b'the', b'a', b'of', b'and'], 10000)

In [42]:
word_to_id = {word: idx for idx, word in enumerate(truncated_vocabulary)}

In [44]:
for word in b"This movie we do itt".split():
    print(word_to_id.get(word) or vocab_size, end='\t')

24	12	108	105	10000	

In [47]:
words=tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
oov_buckets=1000
table = tf.lookup.StaticVocabularyTable(vocab_init, oov_buckets)

In [48]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: id=102106, shape=(1, 4), dtype=int64, numpy=array([[   24,    12,    13, 10053]], dtype=int64)>

In [49]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].repeat().batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [50]:
for X_batch, y_batch in train_set.take(1):
    print(X_batch)
    print(y_batch)

tf.Tensor(
[[   24     7     2 ...     0     0     0]
 [ 1230    83     5 ...    31  4258     0]
 [ 4112     3     1 ...     0     0     0]
 ...
 [   24     7    25 ...     0     0     0]
 [ 1292  3544     7 ...     0     0     0]
 [10928 10687  4552 ...     0     0     0]], shape=(32, 62), dtype=int64)
tf.Tensor([0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 1], shape=(32,), dtype=int64)


In [61]:
valid_set = datasets['test'].take(2000).repeat().batch(32).map(preprocess)
valid_set = valid_set.map(encode_words).prefetch(1)

In [70]:
for x, y in valid_set.take(1):
    print(x,y)

tf.Tensor(
[[  29 1794   17 ...    0    0    0]
 [   5  264  254 ...    0    0    0]
 [  24  323   16 ...    0    0    0]
 ...
 [  15   95  167 ...    0    0    0]
 [ 159    7  340 ...    0    0    0]
 [   5  100   10 ...    2  105    0]], shape=(32, 62), dtype=int64) tf.Tensor([1 0 1 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1], shape=(32,), dtype=int64)


In [64]:
embed_size = 128
model = keras.models.Sequential([
# mask_zero=True means word that mapped as 0 is gonna get neglect in its time step and previous hidden state would be moved
    keras.layers.Embedding(vocab_size + oov_buckets, embed_size, mask_zero=True, input_shape=[None]),
    keras.layers.GRU(64, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation="sigmoid")
])

root_dir = os.path.join(os.curdir, 'my_logs')
tensorboard_callback = keras.callbacks.TensorBoard(root_dir)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         1408000   
_________________________________________________________________
gru_4 (GRU)                  (None, None, 64)          37248     
_________________________________________________________________
gru_5 (GRU)                  (None, 32)                9408      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,454,689
Trainable params: 1,454,689
Non-trainable params: 0
_________________________________________________________________


In [72]:
history = model.fit(train_set, steps_per_epoch = train_size//32,
                    validation_data=valid_set, validation_steps=2000//32,
                    epochs=5, callbacks=[tensorboard_callback])

Train for 781 steps, validate for 62 steps
Epoch 1/5
781/781 [==============================] - 117s 150ms/step - loss: 0.5327 - accuracy: 0.7284 - val_loss: 0.4999 - val_accuracy: 0.7591
Epoch 2/5
781/781 [==============================] - 74s 95ms/step - loss: 0.3502 - accuracy: 0.8581 - val_loss: 0.5479 - val_accuracy: 0.7480
Epoch 3/5
781/781 [==============================] - 70s 89ms/step - loss: 0.2095 - accuracy: 0.9255 - val_loss: 0.6583 - val_accuracy: 0.7329
Epoch 4/5
781/781 [==============================] - 66s 85ms/step - loss: 0.1521 - accuracy: 0.9461 - val_loss: 0.7532 - val_accuracy: 0.7349
Epoch 5/5
781/781 [==============================] - 61s 78ms/step - loss: 0.0965 - accuracy: 0.9683 - val_loss: 1.0095 - val_accuracy: 0.7389


In [82]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(64, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(32)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 128)    1408000     input_7[0][0]                    
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, None)         0           input_7[0][0]                    
__________________________________________________________________________________________________
gru_14 (GRU)                    (None, None, 64)     37248       embedding_7[0][0]                
____________________________________________________________________________________________

In [83]:
history = model.fit(train_set, steps_per_epoch = train_size//32,
                    validation_data=valid_set, validation_steps=2000//32,
                    epochs=3, callbacks=[tensorboard_callback])

Train for 781 steps, validate for 62 steps
Epoch 1/3
781/781 [==============================] - 92s 118ms/step - loss: 0.5313 - accuracy: 0.7242 - val_loss: 0.4972 - val_accuracy: 0.7641
Epoch 2/3
781/781 [==============================] - 46s 58ms/step - loss: 0.3498 - accuracy: 0.8550 - val_loss: 0.5538 - val_accuracy: 0.7485
Epoch 3/3
781/781 [==============================] - 36s 47ms/step - loss: 0.2054 - accuracy: 0.9260 - val_loss: 0.6493 - val_accuracy: 0.7243


## Reusing Pretrained Embedding

In [88]:
TFHUB_CACHE_DIR = os.path.join(os.curdir, 'my_tfhub_cache')
os.environ['TFHUB_CACHE_DIR'] = TFHUB_CACHE_DIR

In [96]:
import tensorflow_hub as hub

model = keras.Sequential([
    # 1 at the end means version 1
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_17 (Dense)             (None, 128)               6528      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 129       
Total params: 48,197,257
Trainable params: 6,657
Non-trainable params: 48,190,600
_________________________________________________________________


In [94]:
for dirpath, dirname, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe.descriptor.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\saved_model.pb
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\assets\tokens.txt
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.data-00000-of-00001
.\my_tfhub_cache\82c4aaf4250ffb09088bd48368ee7fd00e5464fe\variables\variables.index


In [95]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].repeat().batch(batch_size).prefetch(1)
valid_set = datasets['test'].take(2000).repeat().batch(batch_size).prefetch(1)

history = model.fit(train_set, steps_per_epoch=train_size // batch_size,
                    validation_data=valid_set, validation_steps=2000//batch_size, epochs=5)

Train for 781 steps, validate for 62 steps
Epoch 1/5
781/781 [==============================] - 28s 36ms/step - loss: 0.5484 - accuracy: 0.7233 - val_loss: 0.5333 - val_accuracy: 0.7384
Epoch 2/5
781/781 [==============================] - 26s 33ms/step - loss: 0.5146 - accuracy: 0.7490 - val_loss: 0.5265 - val_accuracy: 0.7450
Epoch 3/5
781/781 [==============================] - 26s 33ms/step - loss: 0.5091 - accuracy: 0.7520 - val_loss: 0.5238 - val_accuracy: 0.7475
Epoch 4/5
781/781 [==============================] - 25s 31ms/step - loss: 0.5054 - accuracy: 0.7554 - val_loss: 0.5204 - val_accuracy: 0.7510
Epoch 5/5
781/781 [==============================] - 24s 31ms/step - loss: 0.5022 - accuracy: 0.7562 - val_loss: 0.5168 - val_accuracy: 0.7550
